# Principal-Agent Model

> **Note the following:** 
> 1. This is *not* meant to be an example of an actual **model analysis project**, just an example of how to structure such a project.
> 1. Remember the general advice on structuring and commenting your code
> 1. The `modelproject.py` file includes a function which could be used multiple times in this notebook.

Imports and set magics:

In [1]:
import numpy as np
from scipy import optimize
import sympy as sm

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# local modules
import modelproject

# Model description

# Principal-Agent Model Overview

The Principal-Agent model addresses scenarios where one party (the principal) hires another (the agent) to perform a task. The principal cannot observe the agent's effort directly but can see the outcomes, which are influenced by the agent’s effort. The key challenge is designing a contract that aligns the agent's actions with the principal's interests despite the information asymmetry.

This model explores how contracts can be designed to align the interests of the principal and the agent. It is applicable in employment relationships, contracting scenarios, and any situation where work is delegated under conditions of asymmetric information. This assignment will apply the Principal-Agent model to an insurance context.

## Agent's Utility Function

The agent is a consumer whose utility depends on consumption (c) through the function $v(c)$. Consumption depends on income $M$. The agent has a probability $\pi$ of needing their insurance. If the agent needs their insurance, they incur a loss $L$, and their consumption would be:
$$
c = M - L
$$
With probability $1 - \pi$, the agent does not need their insurance:
$$
c = M
$$

## Principal's Payoff Function

The principal in this case is an insurance company, which sells an insurance policy at a price $\Gamma$ and offers a payment $K$ when the insurance is claimed. The consumer first pays $\Gamma$, hence we can express the consumption probabilities for the principal as follows:

With probability $\pi$:
$$
c = M - \Gamma - L + K
$$

With probability $1 - \pi$:
$$
c = M - \Gamma
$$

The principal offers the contract to the agent with the given $K$ and $\Gamma$. The agent must achieve a higher utility from accepting the contract than their reservation utility, which is given as:
$$
\bar{v} = \pi \cdot v(M - L) + (1 - \pi) \cdot v(M)
$$

The agent is assumed to be risk-averse and the insurance company is profit-maximizing. Therefore, the principal solves:
$$
\max_{\Gamma, K} \pi \cdot (\Gamma - K) + (1 - \pi) \cdot \Gamma
$$
subject to:
$$
\pi \cdot v(M - \Gamma - L + K) + (1 - \pi) \cdot v(M - \Gamma) \geq \bar{v}
$$

This can be simplified by defining $c_1 = M - \Gamma - L + K$ which is the consumption with probability $1 - \pi$, and $c_2 = M - \Gamma$ which is the consumption with probability $\pi$. We can then rewrite the problem as:
$$
\max_{c_1, c_2} M - \pi \cdot L - (1 - \pi) \cdot c_1 - \pi \cdot c_2
$$
subject to:
$$
\pi \cdot v(c_2) + (1 - \pi) \cdot v(c_1) \geq \bar{v}
$$


## Analytical solution

# Solving the Problem

Simplify by Assuming the Constraint is Binding (or adjust the price accordingly so we can solve using Lagrange):**
  The Lagrangian can be set up as:
  $$
  L(c_1, c_2, \lambda) = M - \pi \cdot L - (1 - \pi) c_1 - \pi c_2 + \lambda \left( \pi \cdot v(c_2) + (1 - \pi) \cdot v(c_1) - \bar{v} \right)
  $$

- **First Order Conditions:**
  The partial derivatives of the Lagrangian with respect to $c_1$, $c_2$, and $\lambda$ yield the following system of equations:
  $$
  -(1 - \pi) + \lambda (1 - \pi) v'(c_1) = 0 \quad \Rightarrow \quad v'(c_1) = \frac{1}{\lambda}
  $$
  $$
  -\pi + \lambda \pi v'(c_2) = 0 \quad \Rightarrow \quad v'(c_2) = \frac{1}{\lambda}
  $$

- **Implications of Concavity ($v'' < 0$):**
  Since $v'' < 0$, $v'$ is strictly decreasing. This implies that in the optimal contract, $c_1 = c_2$ (i.e., $K = L$, where due to the binding constraint we can also determine $\Gamma$).
  


# Analytical Setup of the Principal-Agent Model

## Step 1: Define the Utility Functions

The agent's utility function is assumed to be quadratic, represented by:
$$
v(c) = -\frac{1}{2}(c - \bar{c})^2
$$
where $\bar{c}$ is the agent's desired consumption level, typically equivalent to the agent's income in the absence of any losses.

The derivative of this utility function, which is necessary for the Lagrangian formulation, is:
$$
v'(c) = -(c - \bar{c})
$$

## Step 2: Setup the Lagrangian

Given that the constraint is binding (meaning the agent receives exactly their reservation utility), the Lagrangian for this Principal-Agent problem is:
$$
\mathcal{L}(c_1, c_2, \lambda) = M - \pi L - (1 - \pi) c_1 - \pi c_2 + \lambda \left(\pi v(c_2) + (1-\pi) v(c_1) - \bar{v} \right)
$$
where:
- $M$ is the income,
- $\pi$ is the probability of incurring a loss,
- $L$ is the loss amount,
- $c_1$ and $c_2$ are consumptions in scenarios without and with insurance claims, respectively,
- $\lambda$ is the Lagrange multiplier,
- $\bar{v}$ is the agent's reservation utility.

## Step 3: Define the First Order Conditions

To find the optimal contract terms ($c_1$ and $c_2$) and the value of the Lagrange multiplier ($\lambda$), the first-order conditions are derived and solved as follows:
$$
-(1 - \pi) + \lambda(1 - \pi) v'(c_1) = 0 \quad \Rightarrow \quad v'(c_1) = \frac{1}{\lambda}
$$
$$
-\pi + \lambda \pi v'(c_2) = 0 \quad \Rightarrow \quad v'(c_2) = \frac{1}{\lambda}
$$
$$
\pi v(c_2) + (1-\pi) v(c_1) = \bar{v} \quad \text{(the constraint)}
$$

These equations help determine the optimal insurance policy terms that balance the incentives between the insurance company (principal) and the insured (agent), ensuring the agent's willingness to participate.


## Numerical solution

In [2]:
import numpy as np
from scipy.optimize import fsolve

# Define parameters
pi = 0.5  # Probability of needing insurance
M = 100  # Income
L = 50  # Loss
bar_v = -10  # Reservation utility
bar_c = M  # Desired consumption

# Utility function and its derivative
def v(c):
    return -0.5 * (c - bar_c) ** 2

def v_prime(c):
    return -(c - bar_c)

# Define the equations derived from the first order conditions
def equations(p):
    c1, c2, lambda_ = p
    eq1 = -(1 - pi) + lambda_ * (1 - pi) * v_prime(c1)
    eq2 = -pi + lambda_ * pi * v_prime(c2)
    eq3 = pi * v(c2) + (1 - pi) * v(c1) - bar_v
    return (eq1, eq2, eq3)

# Initial guesses for c1, c2, and lambda
initial_guesses = [M, M, 1]
solution = fsolve(equations, initial_guesses)

print(f'Solution: c1 = {solution[0]}, c2 = {solution[1]}, lambda = {solution[2]}')


Solution: c1 = 95.52786404454889, c2 = 95.52786404384028, lambda = 0.2236067967866568


# Further analysis

Make detailed vizualizations of how your model changes with parameter values. 

Try to make an extension of the model. 

# Conclusion

Add concise conclusion. 